# Import libraries and dataset

In [23]:
import pandas as pd
import numpy as np
from datetime import datetime
date_format = "%Y-%m-%d"

In [24]:
all_seasons = pd.read_csv('Datasets/all_seasons.csv')
all_seasons = all_seasons[['datetime', 'conditions']]

In [25]:
all_seasons.head()

,datetime,conditions
0,2000-01-01,Partially cloudy
1,2000-01-02,Clear
2,2000-01-03,Clear
3,2000-01-04,Clear
4,2000-01-05,Clear


# Classify and separate data

In [26]:
classifier = {'Overcast':'overcast', 'Partially cloudy':'partially_cloudy', 'Clear':'clear', 'Rain, Partially cloudy':'rain_partially_cloudy', 'Rain':'rain', 'Rain, Overcast':'rain_overcast'}

all_seasons['condition'] = all_seasons['conditions'].map(classifier)

In [27]:
all_seasons.head()

,datetime,conditions,condition
0,2000-01-01,Partially cloudy,partially_cloudy
1,2000-01-02,Clear,clear
2,2000-01-03,Clear,clear
3,2000-01-04,Clear,clear
4,2000-01-05,Clear,clear


In [28]:
all_seasons = all_seasons[['datetime', 'condition']]

In [29]:
all_seasons.head()

,datetime,condition
0,2000-01-01,partially_cloudy
1,2000-01-02,clear
2,2000-01-03,clear
3,2000-01-04,clear
4,2000-01-05,clear


In [30]:
train_start_date = '2002-01-01'
train_end_date = '2017-12-31'
all_seasons_train = all_seasons.loc[all_seasons['datetime'].between(train_start_date, train_end_date)]
all_seasons_train = all_seasons_train.reset_index()

test_start_date = '2018-01-01'
test_end_date = '2021-12-31'
all_seasons_test = all_seasons.loc[all_seasons['datetime'].between(test_start_date, test_end_date)]
all_seasons_test = all_seasons_test.reset_index()

# Calculate proportions of conditions & Create transition matrix
We will refer to rain is 'R' and no rain as 'N'

In [32]:
# Initialize count variables

# 0: 'clear' - C
# 1: 'partially_cloudy' - PC
# 2: 'overcast' - OV
# 3: 'rain' - R
# 4: 'rain_partially_cloudy' - RPC
# 5: 'rain_overcast' - ROV

C_after_C_count = 0.0
PC_after_C_count = 0.0
OV_after_C_count = 0.0
R_after_C_count = 0.0
RPC_after_C_count = 0.0
ROV_after_C_count = 0.0

C_after_PC_count = 0.0
PC_after_PC_count = 0.0
OV_after_PC_count = 0.0
R_after_PC_count = 0.0
RPC_after_PC_count = 0.0
ROV_after_PC_count = 0.0

C_after_OV_count = 0.0
PC_after_OV_count = 0.0
OV_after_OV_count = 0.0
R_after_OV_count = 0.0
RPC_after_OV_count = 0.0
ROV_after_OV_count = 0.0

C_after_R_count = 0.0
PC_after_R_count = 0.0
OV_after_R_count = 0.0
R_after_R_count = 0.0
RPC_after_R_count = 0.0
ROV_after_R_count = 0.0

C_after_RPC_count = 0.0
PC_after_RPC_count = 0.0
OV_after_RPC_count = 0.0
R_after_RPC_count = 0.0
RPC_after_RPC_count = 0.0
ROV_after_RPC_count = 0.0

C_after_ROV_count = 0.0
PC_after_ROV_count = 0.0
OV_after_ROV_count = 0.0
R_after_ROV_count = 0.0
RPC_after_ROV_count = 0.0
ROV_after_ROV_count = 0.0

In [33]:
all_seasons_train

,index,datetime,condition
0,731,2002-01-01,partially_cloudy
1,732,2002-01-02,rain_partially_cloudy
2,733,2002-01-03,rain_partially_cloudy
3,734,2002-01-04,partially_cloudy
4,735,2002-01-05,partially_cloudy
...,...,...,...
5839,6570,2017-12-27,clear
5840,6571,2017-12-28,clear
5841,6572,2017-12-29,clear
5842,6573,2017-12-30,partially_cloudy


In [34]:
# Count conditions

all_seasons_train['condition_shift'] = all_seasons_train['condition'].shift(-1)

for i in range(len(all_seasons_train)):
    # Current 'clear'
    if all_seasons_train.loc[i, 'condition'] == 'clear' and all_seasons_train.loc[i, 'condition_shift'] == 'clear':
        C_after_C_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'partially_cloudy' and all_seasons_train.loc[i, 'condition_shift'] == 'clear':
        PC_after_C_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'overcast' and all_seasons_train.loc[i, 'condition_shift'] == 'clear':
        OV_after_C_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rain' and all_seasons_train.loc[i, 'condition_shift'] == 'clear':
        R_after_C_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rain_partially_cloudy' and all_seasons_train.loc[i, 'condition_shift'] == 'clear':
        RPC_after_C_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rain_overcast' and all_seasons_train.loc[i, 'condition_shift'] == 'clear':
        ROV_after_C_count += 1
    # Current 'partially_cloudy'
    elif all_seasons_train.loc[i, 'condition'] == 'partially_cloudy' and all_seasons_train.loc[i, 'condition_shift'] == 'partially_cloudy':
        C_after_PC_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'overcast' and all_seasons_train.loc[i, 'condition_shift'] == 'partially_cloudy':
        PC_after_PC_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rain' and all_seasons_train.loc[i, 'condition_shift'] == 'partially_cloudy':
        OV_after_PC_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rain_partially_cloudy' and all_seasons_train.loc[i, 'condition_shift'] == 'partially_cloudy':
        R_after_PC_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rain_overcast' and all_seasons_train.loc[i, 'condition_shift'] == 'partially_cloudy':
        RPC_after_PC_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rain_overcast' and all_seasons_train.loc[i, 'condition_shift'] == 'partially_cloudy':
        ROV_after_PC_count += 1
    # Current 'overcast'
    elif all_seasons_train.loc[i, 'condition'] == 'partially_cloudy' and all_seasons_train.loc[i, 'condition_shift'] == 'overcast':
        C_after_OV_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'overcast' and all_seasons_train.loc[i, 'condition_shift'] == 'overcast':
        PC_after_OV_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rain' and all_seasons_train.loc[i, 'condition_shift'] == 'overcast':
        OV_after_OV_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rain_partially_cloudy' and all_seasons_train.loc[i, 'condition_shift'] == 'overcast':
        R_after_OV_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rain_overcast' and all_seasons_train.loc[i, 'condition_shift'] == 'overcast':
        RPC_after_OV_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rain_overcast' and all_seasons_train.loc[i, 'condition_shift'] == 'overcast':
        ROV_after_OV_count += 1
    # Current 'rain'
    elif all_seasons_train.loc[i, 'condition'] == 'partially_cloudy' and all_seasons_train.loc[i, 'condition_shift'] == 'rain':
        C_after_R_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'overcast' and all_seasons_train.loc[i, 'condition_shift'] == 'rain':
        PC_after_R_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rain' and all_seasons_train.loc[i, 'condition_shift'] == 'rain':
        OV_after_R_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rain_partially_cloudy' and all_seasons_train.loc[i, 'condition_shift'] == 'rain':
        R_after_R_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rain_overcast' and all_seasons_train.loc[i, 'condition_shift'] == 'rain':
        RPC_after_R_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rain_overcast' and all_seasons_train.loc[i, 'condition_shift'] == 'rain':
        ROV_after_R_count += 1
    # Current 'rain_partially_cloudy'
    elif all_seasons_train.loc[i, 'condition'] == 'partially_cloudy' and all_seasons_train.loc[i, 'condition_shift'] == 'rain_partially_cloudy':
        C_after_RPC_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'overcast' and all_seasons_train.loc[i, 'condition_shift'] == 'rain_partially_cloudy':
        PC_after_RPC_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rain' and all_seasons_train.loc[i, 'condition_shift'] == 'rain_partially_cloudy':
        OV_after_RPC_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rain_partially_cloudy' and all_seasons_train.loc[i, 'condition_shift'] == 'rain_partially_cloudy':
        R_after_RPC_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rain_overcast' and all_seasons_train.loc[i, 'condition_shift'] == 'rain_partially_cloudy':
        RPC_after_RPC_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rain_overcast' and all_seasons_train.loc[i, 'condition_shift'] == 'rain_partially_cloudy':
        ROV_after_RPC_count += 1
    # Current 'rain_overcast'
    elif all_seasons_train.loc[i, 'condition'] == 'partially_cloudy' and all_seasons_train.loc[i, 'condition_shift'] == 'rain_overcast':
        C_after_ROV_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'overcast' and all_seasons_train.loc[i, 'condition_shift'] == 'rain_overcast':
        PC_after_ROV_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rain' and all_seasons_train.loc[i, 'condition_shift'] == 'rain_overcast':
        OV_after_ROV_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rain_partially_cloudy' and all_seasons_train.loc[i, 'condition_shift'] == 'rain_overcast':
        R_after_ROV_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rain_overcast' and all_seasons_train.loc[i, 'condition_shift'] == 'rain_overcast':
        RPC_after_ROV_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rain_overcast' and all_seasons_train.loc[i, 'condition_shift'] == 'rain_overcast':
        ROV_after_ROV_count += 1

In [35]:
current_C_total = C_after_C_count + PC_after_C_count + OV_after_C_count + R_after_C_count + RPC_after_C_count + ROV_after_C_count
current_PC_total = C_after_PC_count + PC_after_PC_count + OV_after_PC_count + R_after_PC_count + RPC_after_PC_count + ROV_after_PC_count
current_OV_total = C_after_OV_count + PC_after_OV_count + OV_after_OV_count + R_after_OV_count + RPC_after_OV_count + ROV_after_OV_count
current_R_total =C_after_R_count + PC_after_R_count + OV_after_R_count + R_after_R_count + RPC_after_R_count + ROV_after_R_count
current_RPC_total = C_after_RPC_count + PC_after_RPC_count + OV_after_RPC_count + R_after_RPC_count + RPC_after_RPC_count + ROV_after_RPC_count
current_ROV_total = C_after_ROV_count + PC_after_ROV_count + OV_after_ROV_count + R_after_ROV_count + RPC_after_ROV_count + ROV_after_ROV_count

In [36]:
C_after_C_prob = C_after_C_count / current_C_total
PC_after_C_prob = PC_after_C_count / current_C_total
OV_after_C_prob = OV_after_C_count / current_C_total
R_after_C_prob = R_after_C_count / current_C_total
RPC_after_C_prob = RPC_after_C_count / current_C_total
ROV_after_C_prob = ROV_after_C_count / current_C_total

C_after_PC_prob = C_after_PC_count / current_PC_total
PC_after_PC_prob = PC_after_PC_count / current_PC_total
OV_after_PC_prob = OV_after_PC_count / current_PC_total
R_after_PC_prob = R_after_PC_count / current_PC_total
RPC_after_PC_prob = RPC_after_PC_count / current_PC_total
ROV_after_PC_prob = ROV_after_PC_count / current_PC_total

C_after_OV_prob = C_after_OV_count / current_OV_total
PC_after_OV_prob = PC_after_OV_count / current_OV_total
OV_after_OV_prob = OV_after_OV_count / current_OV_total
R_after_OV_prob = R_after_OV_count / current_OV_total
RPC_after_OV_prob = RPC_after_OV_count / current_OV_total
ROV_after_OV_prob = ROV_after_OV_count / current_OV_total

C_after_R_prob = C_after_R_count / current_R_total
PC_after_R_prob = PC_after_R_count / current_R_total
OV_after_R_prob = OV_after_R_count / current_R_total
R_after_R_prob = R_after_R_count / current_R_total
RPC_after_R_prob = RPC_after_R_count / current_R_total
ROV_after_R_prob = ROV_after_R_count / current_R_total

C_after_RPC_prob = C_after_RPC_count / current_RPC_total
PC_after_RPC_prob = PC_after_RPC_count / current_RPC_total
OV_after_RPC_prob = OV_after_RPC_count / current_RPC_total
R_after_RPC_prob = R_after_RPC_count / current_RPC_total
RPC_after_RPC_prob = RPC_after_RPC_count / current_RPC_total
ROV_after_RPC_prob = ROV_after_RPC_count / current_RPC_total

C_after_ROV_prob = C_after_ROV_count / current_ROV_total
PC_after_ROV_prob = PC_after_ROV_count / current_ROV_total
OV_after_ROV_prob = OV_after_ROV_count / current_ROV_total
R_after_ROV_prob = R_after_ROV_count / current_ROV_total
RPC_after_ROV_prob = RPC_after_ROV_count / current_ROV_total
ROV_after_ROV_prob = ROV_after_ROV_count / current_ROV_total

In [37]:
# Printing our probabilities for 6x6 transition matrix:
print(C_after_C_prob)
print(PC_after_C_prob)
print(OV_after_C_prob)
print(R_after_C_prob)
print(RPC_after_C_prob)
print(ROV_after_C_prob)

print(C_after_PC_prob)
print(PC_after_PC_prob)
print(OV_after_PC_prob)
print(R_after_PC_prob)
print(RPC_after_PC_prob)
print(ROV_after_PC_prob)

print(C_after_OV_prob)
print(PC_after_OV_prob)
print(OV_after_OV_prob)
print(R_after_OV_prob)
print(RPC_after_OV_prob)
print(ROV_after_OV_prob)

print(C_after_R_prob)
print(PC_after_R_prob)
print(OV_after_R_prob)
print(R_after_R_prob)
print(RPC_after_R_prob)
print(ROV_after_R_prob)

print(C_after_RPC_prob)
print(PC_after_RPC_prob)
print(OV_after_RPC_prob)
print(R_after_RPC_prob)
print(RPC_after_RPC_prob)
print(ROV_after_RPC_prob)

print(C_after_ROV_prob)
print(PC_after_ROV_prob)
print(OV_after_ROV_prob)
print(R_after_ROV_prob)
print(RPC_after_ROV_prob)
print(ROV_after_ROV_prob)

0.6896135265700483
0.19927536231884058
0.0008051529790660225
0.033816425120772944
0.07286634460547504
0.0036231884057971015
0.8529411764705882
0.03789592760180995
0.011877828054298642
0.08880090497737557
0.008484162895927601
0.0
0.7641509433962265
0.16037735849056603
0.0
0.04716981132075472
0.02830188679245283
0.0
0.33695652173913043
0.0
0.22826086956521738
0.391304347826087
0.043478260869565216
0.0
0.398406374501992
0.029880478087649404
0.035856573705179286
0.41434262948207173
0.12151394422310757
0.0
0.35833333333333334
0.06666666666666667
0.008333333333333333
0.275
0.2916666666666667
0.0


In [42]:
# Checking that each row in the transition matrix adds up to 1:
print(C_after_C_prob + PC_after_C_prob + OV_after_C_prob + R_after_C_prob + RPC_after_C_prob + ROV_after_C_prob)
print(C_after_PC_prob + PC_after_PC_prob + OV_after_PC_prob + R_after_PC_prob + RPC_after_PC_prob + ROV_after_PC_prob)
print(C_after_OV_prob + PC_after_OV_prob + OV_after_OV_prob + R_after_OV_prob + RPC_after_OV_prob + ROV_after_OV_prob)
print(C_after_R_prob + PC_after_R_prob + OV_after_R_prob + R_after_R_prob + RPC_after_R_prob + ROV_after_R_prob)
print(C_after_RPC_prob + PC_after_RPC_prob + OV_after_RPC_prob + R_after_RPC_prob + RPC_after_RPC_prob + ROV_after_RPC_prob)
print(C_after_ROV_prob + PC_after_ROV_prob + OV_after_ROV_prob + R_after_ROV_prob + RPC_after_ROV_prob + ROV_after_ROV_prob)

1.0
0.9999999999999999
1.0
1.0
1.0
1.0


In [43]:
# Creating the transition matrix:
transition_matrix = [[C_after_C_prob, PC_after_C_prob, OV_after_C_prob, R_after_C_prob, RPC_after_C_prob, ROV_after_C_prob], 
                    [C_after_PC_prob, PC_after_PC_prob, OV_after_PC_prob, R_after_PC_prob, RPC_after_PC_prob, ROV_after_PC_prob],
                    [C_after_OV_prob, PC_after_OV_prob, OV_after_OV_prob, R_after_OV_prob, RPC_after_OV_prob, ROV_after_OV_prob],
                    [C_after_R_prob, PC_after_R_prob, OV_after_R_prob, R_after_R_prob, RPC_after_R_prob, ROV_after_R_prob],
                    [C_after_RPC_prob, PC_after_RPC_prob, OV_after_RPC_prob, R_after_RPC_prob, RPC_after_RPC_prob, ROV_after_RPC_prob],
                    [C_after_ROV_prob, PC_after_ROV_prob, OV_after_ROV_prob, R_after_ROV_prob, RPC_after_ROV_prob, ROV_after_ROV_prob]]
print(transition_matrix)

[[0.6896135265700483, 0.19927536231884058, 0.0008051529790660225, 0.033816425120772944, 0.07286634460547504, 0.0036231884057971015], [0.8529411764705882, 0.03789592760180995, 0.011877828054298642, 0.08880090497737557, 0.008484162895927601, 0.0], [0.7641509433962265, 0.16037735849056603, 0.0, 0.04716981132075472, 0.02830188679245283, 0.0], [0.33695652173913043, 0.0, 0.22826086956521738, 0.391304347826087, 0.043478260869565216, 0.0], [0.398406374501992, 0.029880478087649404, 0.035856573705179286, 0.41434262948207173, 0.12151394422310757, 0.0], [0.35833333333333334, 0.06666666666666667, 0.008333333333333333, 0.275, 0.2916666666666667, 0.0]]


In [44]:
t_array = np.array(transition_matrix)
print(t_array)

[[6.89613527e-01 1.99275362e-01 8.05152979e-04 3.38164251e-02
  7.28663446e-02 3.62318841e-03]
 [8.52941176e-01 3.78959276e-02 1.18778281e-02 8.88009050e-02
  8.48416290e-03 0.00000000e+00]
 [7.64150943e-01 1.60377358e-01 0.00000000e+00 4.71698113e-02
  2.83018868e-02 0.00000000e+00]
 [3.36956522e-01 0.00000000e+00 2.28260870e-01 3.91304348e-01
  4.34782609e-02 0.00000000e+00]
 [3.98406375e-01 2.98804781e-02 3.58565737e-02 4.14342629e-01
  1.21513944e-01 0.00000000e+00]
 [3.58333333e-01 6.66666667e-02 8.33333333e-03 2.75000000e-01
  2.91666667e-01 0.00000000e+00]]


In [47]:
all_seasons_test.head(1)

,index,datetime,condition
0,6575,2018-01-01,clear


First Day of 2018: clear

In [52]:
def predict_weather_six_conditions(test_data):
    state = {0:'clear', 1:'partially_cloudy', 2:'overcast', 3:'rain', 4:'rain_partially_cloudy', 5:'rain_overcast'}
    n = len(test_data) # how many steps to test
    start_state = 0 # 0 = clear
    test_result = test_data.copy()

    prev_state = start_state
    result = []
    while n-1:
        curr_state = np.random.choice([0,1,2,3,4,5], p=t_array[prev_state]) #taking the probability from the transition matrix
        result.append(state[curr_state])
        prev_state = curr_state
        n -= 1

    curr_state = np.random.choice([0,1,2,3,4,5], p=t_array[prev_state]) #taking the probability from the transition matrix
    result.append(state[curr_state])

    test_result['predicted_condition'] = result

    return test_result

def find_accuracy(predicted_result):
    correct_count = 0.0

    for i in range(len(predicted_result)):
        if predicted_result.loc[i, 'condition'] == predicted_result.loc[i, 'predicted_condition']:
            correct_count += 1

    correct_prop = correct_count / len(predicted_result)

    return correct_prop

def run_predictions_return_avg_accuracy(test_data, trial_count):
    accuracy_sum = 0.0
    for i in range(trial_count):
        predicted_result = predict_weather_six_conditions(test_data)
        accuracy = find_accuracy(predicted_result)
        accuracy_sum += accuracy
    avg_accuracy = accuracy_sum / trial_count

    return avg_accuracy

In [54]:
run_predictions_return_avg_accuracy(all_seasons_test, 100)

0.37097878165639975